In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [7]:
%tensorflow_version 1.6

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.6`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [3]:
%cd '/content/gdrive/My Drive/Dacon/Arirang/'
!dir

/content/gdrive/My Drive/Dacon/Arirang
BBAVectors-Oriented-Object-Detection  pixel_1	 sample_submission.csv	train
labels.json			      pixel.csv  simplified_rbox_cnn


In [21]:
!git clone https://github.com/SIAnalytics/simplified_rbox_cnn

Cloning into 'simplified_rbox_cnn'...
remote: Enumerating objects: 350, done.
remote: Total 350 (delta 0), reused 0 (delta 0), pack-reused 350
Receiving objects: 100% (350/350), 4.42 MiB | 3.32 MiB/s, done.
Resolving deltas: 100% (82/82), done.
Checking out files: 100% (326/326), done.


In [4]:
%cd 'simplified_rbox_cnn/'
!dir

/content/gdrive/My Drive/Dacon/Arirang/simplified_rbox_cnn
anchor_generators  exporter_test.py			  protos
box_coders	   export_inference_graph.py		  __pycache__
BUILD		   faster_rcnn_resnet101_coco_11_06_2017  README.md
builders	   figures				  slim
cloud		   g3doc				  test
configs		   grid_tools.py			  train
CONTRIBUTING.md    grid_trainer.py			  trainer.py
core		   grid_train.py			  trainer_test.py
create_dataset.py  inference.py				  train_n_eval.sh
data_decoders	   __init__.py				  train.py
eval.py		   LICENSE				  train.tfrecord
evaluator.py	   matchers				  utils
eval_util.py	   meta_architectures
exporter.py	   models


In [5]:
! protoc protos/*.proto --python_out=.

In [108]:
! python create_dataset.py --src_dir='train' --dst_path='train.tfrecord' --is_include_only_pos


100% 800/800 [02:58<00:00,  4.48it/s]
N of TFRecords: 2988


In [6]:
! python train.py --pipeline_config_path="configs/rbox_cnn_resnet101.config" --train_dir="train"

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W1009 09:12:43.769984 140357319645056 module_wrapper.py:139] From train.py:76: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

INFO:tensorflow:Scale of 0 disables regularizer.
I1009 09:12:44.220999 140357319645056 regularizers.py:98] Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
I1009 09:12:44.221440 140357319645056 regularizers.py:98] Scale of 0 disables regularizer.
Instructions for updating:
Please switch to tf.train.create_global_step
W1009 09:12:44.223084 140357319645056 deprecation.py:323] From /content/gdrive/My Drive/Dacon/Arirang/simplified_rbox_cnn/t

In [80]:
! python inference.py --pipeline_config_path="configs/rbox_cnn_resnet101.config" --ckpt_path="train/model.ckpt" --image_dir='test' --dst_path='train/baseline_120000.csv'

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
 

DOTA geojson 생성

In [24]:
import os
import json
from typing import List
import math
from glob import glob

from tqdm import tqdm
import cv2
import numpy as np
from PIL import Image

In [25]:
NIA_CLASSES = ['background', '소형 선박', '대형 선박', '민간 항공기', '군용 항공기', '소형 승용차', '버스', '트럭', '기차', '크레인', '다리', '정유탱크',
               '댐', '운동경기장', '헬리패드', '원형 교차로']
CLASS_NAMES_EN = ('background', 'small ship', 'large ship', 'civilian aircraft', 'military aircraft', 'small car', 'bus', 'truck', 'train',
        'crane', 'bridge', 'oil tank', 'dam', 'athletic field', 'helipad', 'roundabout')

In [55]:
def convert_labels_to_objects(coords, class_ids, class_names, image_ids, difficult=0, is_clockwise=False):
    objs = list()
    inst_count = 1

    for polygons, cls_id, cls_name, img_id in tqdm(zip(coords, class_ids, class_names, image_ids), desc="converting labels to objects"):
        x1, y1, x2, y2, x3, y3, x4, y4 = polygons
        single_obj = {'properties' : {}}
        single_obj['properties']['bounds_imcoords'] = str(x1)+','+str(y1)+','+str(x2)+','+str(y2)+','+str(x3)+','+str(y3)+','+str(x4)+','+str(y4)
        single_obj['properties']['difficult'] = difficult
        if cls_name in CLASS_NAMES_EN:
            single_obj['properties']['type_id'] = CLASS_NAMES_EN.index(cls_name)
            single_obj['properties']['type_name'] = cls_name
        else:
            continue
        single_obj['properties']['segmentation'] = [[int(p) for p in polygons]]
        single_obj['properties']['iscrowd'] = 0
        single_obj['properties']['image_id'] = img_id
        single_obj['properties']['id'] = inst_count
        inst_count += 1
        objs.append(single_obj)

    print('objects', len(objs))
    return objs

def load_geojsons(filepath):
    jsons = glob(os.path.join(filepath, '*.json'))
    features = []
    for json_path in tqdm(jsons, desc='loading geojson files'):
        with open(json_path) as f:
            data_dict = json.load(f)
        features += data_dict['features']

    obj_coords = list()
    image_ids = list()
    class_indices = list()
    class_names = list()

    for feature in tqdm(features, desc='extracting features'):
        properties = feature['properties']
        image_ids.append(properties['image_id'].replace('PS4', 'PS3')[:-4]+'.png')
        obj_coords.append([float(num) for num in properties['object_imcoords'].split(",")])
        class_indices.append(properties['type_id'])
        class_names.append(properties['type_name'])

    return image_ids, obj_coords, class_indices, class_names

In [69]:
def json2geojson(imageroot: str, geojsonpath: str, destfile, difficult='-1'):
    # set difficult to filter '2', '1', or do not filter, set '-1'
    data_dict = {}
    data_dict['images'] = []
    data_dict['categories'] = []
    data_dict['features'] = []
    for idex, name in enumerate(CLASS_NAMES_EN):
        single_cat = {'id': idex + 1, 'name': name, 'supercategory': name}
        data_dict['categories'].append(single_cat)

    inst_count = 1
    image_id = 1
    with open(destfile, 'w') as f_out:
        img_files, obj_coords, cls_ids, class_names = load_geojsons(geojsonpath)
        img_id_map= {img_file:i+1 for i, img_file in enumerate(list(set(img_files)))}
        image_ids = [img_id_map[img_file] for img_file in img_files]
        objs = convert_labels_to_objects(obj_coords, cls_ids, class_names, img_files, difficult=difficult, is_clockwise=False)
        data_dict['features'].extend(objs)

        for imgfile in tqdm(img_id_map, desc='saving img info'):
            imagepath = os.path.join(imageroot, imgfile)
            img_id = img_id_map[imgfile]
            img = cv2.imread(imagepath)
            height, width, c = img.shape
            single_image = {}
            single_image['file_name'] = imgfile
            single_image['id'] = img_id
            single_image['width'] = width
            single_image['height'] = height
            data_dict['images'].append(single_image)

        json.dump(data_dict, f_out)

In [70]:
%cd '/content/gdrive/My Drive/Dacon/Arirang/'
rootfolder = ''

json2geojson(imageroot=os.path.join(rootfolder, 'train/images'),
                 geojsonpath=os.path.join(rootfolder, 'train/json'),
                 destfile=os.path.join(rootfolder, 'train/labels.json'))

loading geojson files:   2%|▏         | 17/800 [00:00<00:04, 168.16it/s]

/content/gdrive/My Drive/Dacon/Arirang


extracting features: 100%|██████████| 131436/131436 [00:01<00:00, 98176.38it/s]
converting labels to objects: 131436it [00:02, 62488.17it/s]
saving img info:   0%|          | 3/800 [00:00<00:28, 28.28it/s]

objects 130583


saving img info: 100%|██████████| 800/800 [00:28<00:00, 27.81it/s]


In [63]:
%cd '/content/gdrive/My Drive/Dacon/Arirang/'
img_files, obj_coords, cls_ids, class_names = load_geojsons('train/json')

loading geojson files:   0%|          | 0/800 [00:00<?, ?it/s]

/content/gdrive/My Drive/Dacon/Arirang


extracting features: 100%|██████████| 131436/131436 [00:00<00:00, 172352.33it/s]


토론에 올라온 geojson

In [103]:
import os
%cd '/content/gdrive/My Drive/Dacon/Arirang/'
path = './train/json/'
files = os.listdir(path)
print('json 파일 총 개수 :', len(files))
files[:5]

/content/gdrive/My Drive/Dacon/Arirang
json 파일 총 개수 : 800


['OBJ04053_PS3_K3A_NIA0170.json',
 'OBJ03758_PS3_K3A_NIA0157.json',
 'OBJ04403_PS3_K3A_NIA0289.json',
 'OBJ05076_PS3_K3A_NIA0331.json',
 'OBJ03789_PS3_K3A_NIA0158.json']

In [104]:
import json

all_json = None

for file in files :
    json_data = json.load(open(path + file))

    if all_json == None : all_json = json_data
    else : all_json['features'].extend(json_data['features'])
with open('./labels.json', 'w') as outfile:
    json.dump(all_json, outfile, indent='\t')
!ls

BBAVectors-Oriented-Object-Detection  pixel_1	 sample_submission.csv	train
labels.json			      pixel.csv  simplified_rbox_cnn


https://medium.com/@postor/train-dota-dataset-with-yolov3-a38e34f7f479
==========

In [81]:
%cd '/content/gdrive/My Drive/Dacon/Arirang/'

/content/gdrive/My Drive/Dacon/Arirang


In [84]:
!git clone https://github.com/yijingru/BBAVectors-Oriented-Object-Detection

Cloning into 'BBAVectors-Oriented-Object-Detection'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 94 (delta 45), reused 69 (delta 24), pack-reused 0
Unpacking objects: 100% (94/94), done.


In [86]:
!git clone https://github.com/CAPTAIN-WHU/DOTA_devkit

Cloning into 'DOTA_devkit'...
remote: Enumerating objects: 139, done.
remote: Total 139 (delta 0), reused 0 (delta 0), pack-reused 139
Receiving objects: 100% (139/139), 59.28 MiB | 13.41 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [96]:
%cd '/content/gdrive/My Drive/Dacon/Arirang/BBAVectors-Oriented-Object-Detection/'

/content/gdrive/My Drive/Dacon/Arirang/BBAVectors-Oriented-Object-Detection


In [90]:
!dir

datasets    draw_loss.py  func_utils.py  loss.py  models  README.md  train.py
decoder.py  eval.py	  imgs		 main.py  nms.py  test.py


In [102]:
!python main.py --data_dir dataPath --epochs 80 --batch_size 16 --dataset dota --phase train

Traceback (most recent call last):
  File "main.py", line 6, in <module>
    from datasets.dataset_hrsc import HRSC
  File "/content/gdrive/My Drive/Dacon/Arirang/BBAVectors-Oriented-Object-Detection/datasets/dataset_hrsc.py", line 6, in <module>
    from .DOTA_devkit.hrsc_evaluation_task1 import voc_eval
ModuleNotFoundError: No module named 'datasets.DOTA_devkit.hrsc_evaluation_task1'


In [93]:
%cd '/content/gdrive/My Drive/Dacon/Arirang/BBAVectors-Oriented-Object-Detection/datasets/DOTA_devkit'

/content/gdrive/My Drive/Dacon/Arirang/BBAVectors-Oriented-Object-Detection/datasets/DOTA_devkit


In [94]:
!sudo apt-get install swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 6 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 2s (492 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (Th

In [95]:
!swig -c++ -python polyiou.i
!python setup.py build_ext --inplace

running build_ext
building '_polyiou' extension
creating build
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -c polyiou_wrap.cxx -o build/temp.linux-x86_64-3.6/polyiou_wrap.o
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -c polyiou.cpp -o build/temp.linux-x86_64-3.6/polyiou.o
polyiou.cpp: In function ‘double intersectArea(Point, Point, Point, Point)’:
polyiou.cpp:88:5: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
     if(s1*s2==-1) res=-res;return res;
     ^~
polyiou.cpp:88:28: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘if’
     if(s1*s2==-1) res=-res;return res;
                            ^~~~~~
x